In [1]:
from machinerie import Circuit, draw404

draw404()

# Challenge 3 : Algorithme de Grover _cassé_ (2/2)

Notre ingénieur s'est trompé lorsqu'il a programmé l'algorithme, il a oublié des $H$... Comment faire pour récupérer le drapeau ? 

Soit un drapeau "00101011", le circuit complet s'écrit : 

In [2]:
from machinerie import create_grover

flag = [0, 0, 1, 0, 1, 0, 1, 1]
n = len(flag)

grover = create_grover(flag, range(n), range(n))
grover.draw()

┌─────┐┌───┐┌──────┐┌───┐
q_0: ┤0    ├┤ H ├┤0     ├┤ H ├
     │     │├───┤│      │├───┤
q_1: ┤1    ├┤ H ├┤1     ├┤ H ├
     │     │├───┤│      │├───┤
q_2: ┤2    ├┤ H ├┤2     ├┤ H ├
     │     │├───┤│      │├───┤
q_3: ┤3    ├┤ H ├┤3     ├┤ H ├
     │  zf │├───┤│  Zor │├───┤
q_4: ┤4    ├┤ H ├┤4     ├┤ H ├
     │     │├───┤│      │├───┤
q_5: ┤5    ├┤ H ├┤5     ├┤ H ├
     │     │├───┤│      │├───┤
q_6: ┤6    ├┤ H ├┤6     ├┤ H ├
     │     │├───┤│      │├───┤
q_7: ┤7    ├┤ H ├┤7     ├┤ H ├
     └─────┘└───┘└──────┘└───┘

In [3]:
full_circuit = Circuit(n)
full_circuit.h(range(n))
full_circuit.compose(grover, inplace=True)
full_circuit.compose(grover, inplace=True)
full_circuit.compose(grover, inplace=True)
full_circuit.compose(grover, inplace=True)
# full_circuit.draw()
results = full_circuit.get_measure()
drapeau = sorted(results, key=lambda x: x[1], reverse=True)[0]
print(f"""
Drapeau : {drapeau[::-1]}
Probabilité : {results[drapeau]}
""")


Drapeau : 00101011
Probabilité : 0.2685546875



À une inversion près, on retrouve notre drapeau en 4 coups, ... quand le circuit est bien implémenté. Ce n'est malheureusement pas le cas pour le circuit sur nos serveurs, il manque au moins 2 $H$ par colonne...

Par exemple : 

In [4]:
grover = create_grover(flag, range(n - 2), range(n - 2))
grover.draw()

┌─────┐┌───┐┌──────┐┌───┐
q_0: ┤0    ├┤ H ├┤0     ├┤ H ├
     │     │├───┤│      │├───┤
q_1: ┤1    ├┤ H ├┤1     ├┤ H ├
     │     │├───┤│      │├───┤
q_2: ┤2    ├┤ H ├┤2     ├┤ H ├
     │     │├───┤│      │├───┤
q_3: ┤3    ├┤ H ├┤3     ├┤ H ├
     │  zf │├───┤│  Zor │├───┤
q_4: ┤4    ├┤ H ├┤4     ├┤ H ├
     │     │├───┤│      │├───┤
q_5: ┤5    ├┤ H ├┤5     ├┤ H ├
     │     │└───┘│      │└───┘
q_6: ┤6    ├─────┤6     ├─────
     │     │     │      │     
q_7: ┤7    ├─────┤7     ├─────
     └─────┘     └──────┘

Pour couronner le tout, il n'y a qu'une passe qui a été implémentée.

**Votre mission : récupérer le drapeau.**

Vous avez accès à 3 paramètres : 
- l'entrée
- les positions des $H$ entre $Z_f$ et $Z_\text{OR}$ - vous avez le droit d'en poser $\leq n-2$
- les positions des $H$ après $Z_\text{OR}$ - vous avez le droit d'en poser $\leq n-2$

à travers la fonction `test_flag_grover` (j'utilise exactement la même fonction côté API).


Pour éviter le brute force sur le CTFd directement, **vous devrez reproduire la procédure deux fois, pour récupérer deux drapeaux de 12 bits chacuns,** le drapeau final sera `404CTF{premier_flag+deuxième_flag}`, par exemple : `404CTF{0101010101010101010101010}`

Pour éviter l'explosion de votre ordinateur lors de l'appel à `get_flat_unitary`, vous utiliserez des angles pour m'envoyer votre entrée. À partir d'une liste de $n*3$ flottants, je construis n'importe quel état d'entré avec des portes $U$. Les angles sont ceux de la sphère de Bloch. Vous avez l'implémentation dans `Circuit` : `Circuit.from_angles()`.

Par exemple deux Hadamards : 

In [5]:
from math import pi

# theta_0, phi_0, lambda_0, theta_1, ...
angles = [pi / 2, 0, pi, pi / 2, 0, pi]
qc = Circuit.from_angles(angles)
qc.draw_qubits()

<IPython.core.display.Latex object>

In [6]:
from machinerie import create_zf, test_flag_grover
import json

flag = [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]
n = len(flag)
print("flag = ", flag, len(flag))

zf = create_zf(flag)
zf.draw()


flag =  [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1] 12


┌───┐ 1-Z ┌───┐
 q_0: ┤ X ├──■──┤ X ├
      └───┘  │  └───┘
 q_1: ───────■───────
      ┌───┐  │  ┌───┐
 q_2: ┤ X ├──■──┤ X ├
      └───┘  │  └───┘
 q_3: ───────■───────
      ┌───┐  │  ┌───┐
 q_4: ┤ X ├──■──┤ X ├
      └───┘  │  └───┘
 q_5: ───────■───────
      ┌───┐  │  ┌───┐
 q_6: ┤ X ├──■──┤ X ├
      └───┘  │  └───┘
 q_7: ───────■───────
      ┌───┐  │  ┌───┐
 q_8: ┤ X ├──■──┤ X ├
      └───┘  │  └───┘
 q_9: ───────■───────
      ┌───┐  │  ┌───┐
q_10: ┤ X ├──■──┤ X ├
      └───┘  │  └───┘
q_11: ───────■───────

In [80]:
offset = 6
angles = [pi / 2, 0, 0] * (n - offset) + [0, 0, 0] * offset

print(len(angles) / 3)
for i in range(n):
    pass

qc = Circuit.from_angles(angles)
qc.draw_qubits()
# qc.draw()

12.0


<IPython.core.display.Latex object>

In [83]:
hadamard_middle = range(6, n)
hadamard_end = range(6, n)

test = create_grover(flag, hadamard_middle, hadamard_end)
test.draw()

┌──────┐     ┌───────┐     
 q_0: ┤0     ├─────┤0      ├─────
      │      │     │       │     
 q_1: ┤1     ├─────┤1      ├─────
      │      │     │       │     
 q_2: ┤2     ├─────┤2      ├─────
      │      │     │       │     
 q_3: ┤3     ├─────┤3      ├─────
      │      │     │       │     
 q_4: ┤4     ├─────┤4      ├─────
      │      │     │       │     
 q_5: ┤5     ├─────┤5      ├─────
      │   zf │┌───┐│   Zor │┌───┐
 q_6: ┤6     ├┤ H ├┤6      ├┤ H ├
      │      │├───┤│       │├───┤
 q_7: ┤7     ├┤ H ├┤7      ├┤ H ├
      │      │├───┤│       │├───┤
 q_8: ┤8     ├┤ H ├┤8      ├┤ H ├
      │      │├───┤│       │├───┤
 q_9: ┤9     ├┤ H ├┤9      ├┤ H ├
      │      │├───┤│       │├───┤
q_10: ┤10    ├┤ H ├┤10     ├┤ H ├
      │      │├───┤│       │├───┤
q_11: ┤11    ├┤ H ├┤11     ├┤ H ├
      └──────┘└───┘└───────┘└───┘

In [98]:
hadamard_middle = range(0, n - 6)
hadamard_end = range(0, n - 6)

results = test_flag_grover(flag, qc, hadamard_middle, hadamard_end)
drapeau = sorted(results, key=lambda x: x[1], reverse=True)
print(drapeau)

str_flag = "".join([str(e) for e in flag])
if str_flag in results:
    print(drapeau.index("".join([str(e) for e in flag])))
else:
    print("Pas de drapeau trouvé")

drapeau = drapeau[0]
print(
    f"""
Drapeau : {drapeau[::-1]}
Probabilité : {results[drapeau]}
"""
) #type: ignore

['000000101010', '000000000010', '000000001010', '000000011100', '000000100011', '000000101001', '000000101111', '000000110000', '000000110011', '000000001100', '000000110001', '000000111001', '000000111110', '000000100101', '000000101000', '000000011001', '000000110101', '000000011110', '000000101110', '000000001101', '000000000011', '000000000000', '000000101011', '000000110100', '000000010100', '000000110110', '000000011011', '000000111111', '000000000100', '000000111010', '000000001000', '000000001011', '000000010101', '000000011101', '000000101101', '000000011111', '000000010000', '000000000111', '000000100100', '000000001111', '000000000110', '000000000001', '000000111100', '000000001110', '000000010111', '000000011010', '000000111101', '000000010011', '000000100000', '000000010110', '000000101100', '000000111011', '000000100010', '000000100111', '000000001001', '000000000101', '000000111000', '000000100110', '000000110010', '000000100001', '000000110111', '000000011000', '000000


En appelant l'API, vous obtiendrez une mesure. Pour éviter la surcharge, je mesure à chaque fois sur 1000 essais. ***Le brute force de l'API est évidemment toujours interdit, vous êtes sensé pouvoir trouver le drapeau avec moins de $30$ essais ($5$ si vous n'êtes pas trop malchanceux).*** 

In [ ]:
import requests
import json

data = {
    "input_qubits": angles,
    "hadamard_middle": list(range(10)),
    "hadamard_end": list(range(10)),
}

# Première partie du drapeau :
url = "https://causapscal-des-profondeurs.404ctf.fr/grover/1"

# Seconde partie du drapeau :
# url = "https://causapscal-des-profondeurs.404ctf.fr/grover/2"

headers = {"Content-Type": "application/json", "Accept": "application/json"}
response = requests.post(url, json=data, headers=headers)

print(json.loads(response.content)["message"])